In [1]:
# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# Data imports
import pandas as pd
import numpy as np
from tqdm import tqdm

# Visualization imports
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 4]
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns


# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score, train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('Data/bank-full.csv', sep=';')

### **Bank Client Data**

The dataset contains information about bank clients. Here is the information organized in a table format:

#### **Client Information**
| Attribute | Description                   | Type        | Possible Values                                                                 |
|-----------|-------------------------------|-------------|---------------------------------------------------------------------------------|
| **Age**       | Client age                    | Numeric     |                                                                                 |
| **Job**       | Type of job                   | Categorical | *admin.*, *unknown*, *unemployed*, *management*, *housemaid*, *entrepreneur*, *student*, *blue-collar*, *self-employed*, *retired*, *technician*, *services* |
| **Marital**   | Marital status                | Categorical | *married*, *divorced (means divorced or widowed)*, *single*                                                |
| **Education** | Education level               | Categorical | *unknown*, *secondary*, *primary*, *tertiary*                                   |
| **Default**   | Has credit in default?        | Binary      | *yes*, *no*                                                                     |
| **Balance**   | Average yearly balance, euros | Numeric     |                                                                                 |
| **Housing**   | Has housing loan?             | Binary      | *yes*, *no*                                                                     |
| **Loan**      | Has personal loan?            | Binary      | *yes*, *no*                                                                     |

#### **Related with the last contact of the current campaign**
| Attribute | Description                   | Type        | Possible Values                                                                 |
|-----------|-------------------------------|-------------|---------------------------------------------------------------------------------|
| **Contact**   | Contact communication type    | Categorical | *unknown*, *telephone*, *cellular*                                             |
| **Day**       | Last contact day of the month | Numeric     |                                                                                 |
| **Month**     | Last contact month of year    | Categorical | *jan*, *feb*, *mar*, *apr*, *may*, *jun*, *jul*, *aug*, *sep*, *oct*, *nov* *dec*                                                |
| **Duration**  | Last contact duration, seconds| Numeric     |                                                                                 |

#### **Other attributes**
| Attribute | Description                   | Type        | Possible Values                                                                 |
|-----------|-------------------------------|-------------|---------------------------------------------------------------------------------|
| **Campaign**  | Number of contacts in campaign| Numeric     |                                                                                 |
| **Pdays**     | Days since last contact       | Numeric     | -1 means not previously contacted                                             |
| **Previous**  | Number of contacts prior to this campaign | Numeric |                                                                         |
| **Poutcome**  | Outcome of previous marketing campaign  | Categorical | *unknown*, *other*, *failure*, *success*                                       |

#### **Output variable (desired target)**
| Attribute | Description                   | Type        | Possible Values                                                                 |
|-----------|-------------------------------|-------------|---------------------------------------------------------------------------------|
| **Y**  | has the client subscribed a term deposit?| binary     |  *yes*, *no*  |


In [6]:
# create report
profile = ProfileReport(
    df,
    title='Bank Marketing Profiling Report',
    interactions=None,
    duplicates=None
)

# profile.to_notebook_iframe()
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Define columns
ordinal_feature = ['pdays']
nominal_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
numeric_features = ['age', 'balance', 'day', 'duration', 'campaign', 'previous']

# Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(categories=[sorted(df.pdays.unique())]), ordinal_feature),
        ('nominal', OneHotEncoder(drop='first'), nominal_features),
        ('numerical', StandardScaler(), numeric_features)
    ],
)

# Encode Y
# Define the mapping
mapping = {'yes': 1, 'no': 0}
# Apply mapping to the target column
df['y'] = df['y'].map(mapping)

# SPLIT data
X_train, X_test, y_train, y_test = train_test_split(df.drop('y', axis=1), df['y'], test_size=0.2, stratify=df['y'], random_state=42)

In [8]:
# Create a list of model objects
models = [
    ('Random Forest', RandomForestClassifier()),
    ('XGBoost', XGBClassifier()),
    ('Logistic Regression', LogisticRegression()),
    ('SVC', SVC()),
    ('K Neighbors', KNeighborsClassifier())
]

In [9]:
# Define the KFold for cross-validation
split = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in tqdm(models):
    # Compose data preprocessing and model into a single pipeline
    steps = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Compute cross-validation accuracy for each model
    cv_results = cross_val_score(steps, X_train, y_train, cv=split)
    
    # Outputs rounded to 4 decimal places
    min_score = round(cv_results.min(), 4)
    max_score = round(cv_results.max(), 4)
    mean_score = round(cv_results.mean(), 4)
    std_dev = round(cv_results.std(), 4)
    
    print(f"[{name}] Cross Validation Accuracy Score: {mean_score} +/- {std_dev} (std) min: {min_score}, max: {max_score}")

 20%|██        | 1/5 [01:28<05:52, 88.06s/it]

[Random Forest] Cross Validation Accuracy Score: 0.9049 +/- 0.0038 (std) min: 0.8996, max: 0.9107


 40%|████      | 2/5 [01:33<01:58, 39.52s/it]

[XGBoost] Cross Validation Accuracy Score: 0.9045 +/- 0.0027 (std) min: 0.8995, max: 0.907


 60%|██████    | 3/5 [01:35<00:44, 22.39s/it]

[Logistic Regression] Cross Validation Accuracy Score: 0.9013 +/- 0.0031 (std) min: 0.8973, max: 0.9054


 80%|████████  | 4/5 [03:16<00:53, 53.32s/it]

[SVC] Cross Validation Accuracy Score: 0.8829 +/- 0.0002 (std) min: 0.8825, max: 0.8832


100%|██████████| 5/5 [03:50<00:00, 46.16s/it]

[K Neighbors] Cross Validation Accuracy Score: 0.8922 +/- 0.0047 (std) min: 0.8843, max: 0.8976


First three algorithms perform better than the other two. They are worth the shot so I'll try to finu tune each of them.

In [11]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

## Random Forest

In [12]:
from scipy.stats import randint, uniform

In [10]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

In [13]:
# Define the hyperparameter grid for randomized search
param_dist = {
    'classifier__n_estimators': randint(100, 500),
    'classifier__max_features': ['auto', 'sqrt', 'log2', None],
    'classifier__max_depth': [None] + list(randint(5, 50).rvs(size=10)),
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False],
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__class_weight': [None, 'balanced', 'balanced_subsample'],
    'classifier__max_samples': [None] + list(uniform(0.1, 1.0).rvs(size=10)),
}

In [14]:
# Create a randomized search object
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=100,  # Number of random combinations to try
    scoring='accuracy',  # Choose an appropriate scoring metric
    cv=5,  # Number of cross-validation folds
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [15]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('ordinal',
                                                                               OrdinalEncoder(categories=[[-1,
                                                                                                           1,
                                                                                                           2,
                                                                                                           3,
                                                                                                           4,
                                                                                                           5,
                                                                                                           6,
                                                                                                           7,
                                                                                                           8,
                                                                                                           9,
                                                                                                           10,
                                                                                                           12,
                                                                                                           13,
                                                                                                           14,
                                                                                                           15,
                                                                                                           17,
                                                                                                           18,
                                                                                                           19,
                                                                                                           20,
                                                                                                           21,
                                                                                                           22,
                                                                                                           24,
                                                                                                           25,
                                                                                                           26,
                                                                                                           27,
                                                                                                           28,
                                                                                                           29,
                                                                                                           30,
                                                                                                           31,
                                                                                                           32, ...]]),
                                                                               ['pdays']),
                                                                              ('nominal',
                                                                               OneHotEncoder(drop='first'),
                                                                               ['job',
                                                                                'marital',
                           

In [16]:
best_model = random_search.best_estimator_

In [17]:
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ordinal',
                                                  OrdinalEncoder(categories=[[-1,
                                                                              1,
                                                                              2,
                                                                              3,
                                                                              4,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9,
                                                                              10,
                                                                              12,
                                                                              13,
                                                                              14,
                                                                              15,
                                                                              17,
                                                                              18,
                                                                              19,
                                                                              20,
                                                                              21,
                                                                              22,
                                                                              24,
                                                                              25,
                                                                              26,
                                                                              27,
                                                                              28,
                                                                              29,
                                                                              30,
                                                                              31,
                                                                              32, ...]]),
                                                  ['pdays']),
                                                 ('nominal',
                                                  OneHotEncoder(drop='first'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'poutcome']),
                                                 ('numerical', StandardScaler(),
                                                  ['age', 'balance', 'day',
                                                   'duration', 'campaign',
                                                   'previous'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=35, max_features=None,
                                        max_samples=0.37225571012980074,
                                        min_samples_split=10,
                                        n_estimators=473))])

In [18]:
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9068893066460245


## XGBoost

In [22]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**param))
])

In [21]:
param = {'gpu_id':0, 'tree_method':'gpu_hist'}

In [23]:
param_dist = {
    'classifier__n_estimators': randint(100, 500),
    'classifier__max_depth': randint(3, 10),
    'classifier__learning_rate': uniform(0.01, 0.3),
    'classifier__subsample': uniform(0.5, 0.5),
    'classifier__colsample_bytree': uniform(0.5, 0.5),
    'classifier__gamma': uniform(0, 5),
    'classifier__min_child_weight': randint(1, 10),
}


In [24]:
# Create a randomized search object inside the pipeline
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=100,  # Number of random combinations to try
    scoring='accuracy',  # Choose an appropriate scoring metric
    cv=5,  # Number of cross-validation folds
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [25]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('ordinal',
                                                                               OrdinalEncoder(categories=[[-1,
                                                                                                           1,
                                                                                                           2,
                                                                                                           3,
                                                                                                           4,
                                                                                                           5,
                                                                                                           6,
                                                                                                           7,
                                                                                                           8,
                                                                                                           9,
                                                                                                           10,
                                                                                                           12,
                                                                                                           13,
                                                                                                           14,
                                                                                                           15,
                                                                                                           17,
                                                                                                           18,
                                                                                                           19,
                                                                                                           20,
                                                                                                           21,
                                                                                                           22,
                                                                                                           24,
                                                                                                           25,
                                                                                                           26,
                                                                                                           27,
                                                                                                           28,
                                                                                                           29,
                                                                                                           30,
                                                                                                           31,
                                                                                                           32, ...]]),
                                                                               ['pdays']),
                                                                              ('nominal',
                                                                               OneHotEncoder(drop='first'),
                                                                               ['job',
                                                                                'marital',
                           

In [26]:
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

print(best_params)
best_estimator

{'classifier__colsample_bytree': 0.7558787879651307, 'classifier__gamma': 2.201987640424965, 'classifier__learning_rate': 0.1703765954667891, 'classifier__max_depth': 5, 'classifier__min_child_weight': 2, 'classifier__n_estimators': 355, 'classifier__subsample': 0.8207415174131929}


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ordinal',
                                                  OrdinalEncoder(categories=[[-1,
                                                                              1,
                                                                              2,
                                                                              3,
                                                                              4,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9,
                                                                              10,
                                                                              12,
                                                                              13,
                                                                              14,
                                                                              15,
                                                                              17,
                                                                              18,
                                                                              19,
                                                                              20,
                                                                              21,
                                                                              22,
                                                                              24,
                                                                              25,
                                                                              26,
                                                                              27,
                                                                              28,
                                                                              29,
                                                                              30,
                                                                              31,
                                                                              32, ...]]),
                                                  ['pdays']),
                                                 ('nominal',
                                                  OneHotEncoder(drop='first'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'poutcome']),
                                                 ('numerical', StandardS...
                               gpu_id=0, grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.1703765954667891, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=5,
                               max_leaves=None, min_child_weight=2, missing=nan,
                               monotone_constraints=None, n_estimators=355,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [27]:
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9068893066460245


In [28]:
# Generate and print the classification report
y_pred = best_estimator.predict(X_test)
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95      7985
           1       0.64      0.47      0.54      1058

    accuracy                           0.91      9043
   macro avg       0.79      0.72      0.75      9043
weighted avg       0.90      0.91      0.90      9043



In [29]:
# Create an interactive heatmap using Plotly
fig = px.imshow(cm, x=best_estimator.classes_, y=best_estimator.classes_,
                labels=dict(x="Predicted Labels", y="True Labels", color="Count"),
                title="Confusion Matrix")
fig.update_layout(width=800, height=500)

# Set count values on tiles
fig.update_traces(texttemplate='%{text}', text=cm.tolist())

# Display the interactive plot
fig.show()

NameError: name 'cm' is not defined

In [30]:
# Get feature importances
importances = best_estimator.feature_importances_

# Get feature names
feature_names = list(ordinal_feature+preprocessor.transformers_[1][1].get_feature_names_out().tolist()+numeric_features)  # Replace with your actual feature names

# Create a DataFrame with feature names and importances
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Create an interactive bar plot using Plotly
fig = px.bar(importance_df, x='Importance', y='Feature', orientation='h', title='Feature Importances from XGBoost Model')

# Display the interactive plot
fig.show()

AttributeError: 'Pipeline' object has no attribute 'feature_importances_'

In [59]:
import shap

In [60]:
# Create an explainer using SHAP
explainer = shap.Explainer(best_estimator)

# Calculate SHAP values for the testing data
shap_values = explainer.shap_values(X_test)

# Visualize feature importances using a summary plot
shap.summary_plot(shap_values, X_test, feature_names=feature_names)